## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page: [Human phenotype ontology entity resolver](https://aws.amazon.com/marketplace/pp/prodview-tevdjtpf7wj2w)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Pipeline for Human Phenotype Ontology (HPO) Sentence Entity Resolver

- **Model**: `hpo_vdb_resolver`
- **Model Description**: This pretrained pipeline extracts human phenotype entities from clinical text and maps them to their corresponding Human Phenotype Ontology (HPO) codes.

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [ ]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

In [ ]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

In [4]:
model_name = "hpo-vdb-resolver"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)


### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [ ]:
# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

### Initial Setup

In [8]:
docs = [
    """HPI: A 69-year-old white female with a history of depression, anxiety, admitted to the ABCD Hospital on February 6, 2007, for shortness of breath. The patient was consulted by Psychiatry for anxiety. I know this patient from a previous consult. During this recent admission, she was given Ativan 0.25 mg on a p.r.n. basis with relief after one to two hours. The patient was seen by Abc, MD, and Def, Ph.D. PAST MEDICAL HISTORY: The patient has a history of hypertension, hypothyroidism, cholelithiasis, Port-A-Cath placement, and hydronephrosis.""",
    """ A 5-month-old boy brought by his parents because of 2 days of cough. Mother took him when cough started 2 days go to Clinic. But cough got worse and he also started having fever yesterday at night. Mother did not measure it. REVIEW OF SYSTEMS: No vomiting. No diarrhea. No skin rash. No cyanosis. """]

sample_text = """She is followed by Dr. X in our office and has a history of severe tricuspid regurgitation. On 05/12/08, preserved left and right ventricular systolic function, aortic sclerosis with apparent mild aortic stenosis. She has previously had a Persantine Myoview nuclear rest-stress test scan completed at ABCD Medical Center in 07/06 that was negative. She has had significant mitral valve regurgitation in the past being moderate, but on the most recent echocardiogram on 05/12/08, that was not felt to be significant. She does have a history of significant hypertension in the past. She has had dizzy spells and denies clearly any true syncope. She has had bradycardia in the past from beta-blocker therapy."""

### JSON

In [9]:
input_json_data = {"text": sample_text}
data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data["predictions"][0])

,begin,end,ner_chunk,ner_label,ner_confidence,concept_code,resolution,score,aux,all_codes,all_resolutions,all_score
0,67,89,tricuspid regurgitation,HP,0.9826,HP:0005180,tricuspid regurgitation,1.0,"[Fyler:1161||SNOMEDCT_US:111287006||UMLS:C0040961, Fyler:1161||SNOMEDCT_US:111287006||UMLS:C0040961, Fyler:1161||SNOMEDCT_US:111287006||UMLS:C0040961, EPCC:06.01.92||ICD-10:Q22.4||SNOMEDCT_US:49915006||UMLS:C0040963, EPCC:06.01.00||UMLS:C4025753]","[HP:0005180, HP:0005180, HP:0005180, HP:0010446, HP:0001702]","[tricuspid regurgitation, tricuspid valve regurgitation, tricuspid insufficiency, tricuspid stenosis, abnormality of the tricuspid valve]","[1.0000003576278687, 0.9726435542106628, 0.8386023640632629, 0.8336495161056519, 0.8167619705200195]"
1,197,211,aortic stenosis,HP,0.8053,HP:0001650,aortic stenosis,1.0,"[Fyler:1411||SNOMEDCT_US:60573004||UMLS:C0003507, Fyler:1411||SNOMEDCT_US:60573004||UMLS:C0003507, Fyler:1411||SNOMEDCT_US:60573004||UMLS:C0003507, SNOMEDCT_US:204368006||UMLS:C0340375, SNOMEDCT_US:204368006||UMLS:C0340375]","[HP:0001650, HP:0001650, HP:0001650, HP:0001682, HP:0001682]","[aortic stenosis, aortic valve stenosis, valvular aortic stenosis, subvalvular aortic stenosis, subaortic stenosis]","[0.9999999403953552, 0.9627023935317993, 0.9433531165122986, 0.8516278266906738, 0.8179107904434204]"
2,373,398,mitral valve regurgitation,HP,0.91050005,HP:0001653,mitral valve regurgitation,1.0,"[Fyler:1151||SNOMEDCT_US:48724000||UMLS:C0026266||UMLS:C3551535, Fyler:1151||SNOMEDCT_US:48724000||UMLS:C0026266||UMLS:C3551535, SNOMEDCT_US:60234000||UMLS:C0003504, SNOMEDCT_US:91434003||UMLS:C0034088, -]","[HP:0001653, HP:0001653, HP:0001659, HP:0010444, HP:0034376]","[mitral valve regurgitation, mitral regurgitation, aortic valve regurgitation, pulmonary valve regurgitation, atrioventricular valve regurgitation]","[0.9999997615814209, 0.9782090187072754, 0.8417404890060425, 0.838316798210144, 0.8361339569091797]"
3,555,566,hypertension,HP,0.9848,HP:0000822,hypertension,1.0,"[SNOMEDCT_US:24184005||SNOMEDCT_US:38341003||UMLS:C0020538||UMLS:C0497247, SNOMEDCT_US:24184005||SNOMEDCT_US:38341003||UMLS:C0020538||UMLS:C0497247, SNOMEDCT_US:24184005||SNOMEDCT_US:38341003||UMLS:C0020538||UMLS:C0497247, SNOMEDCT_US:24184005||SNOMEDCT_US:38341003||UMLS:C0020538||UMLS:C0497247, -]","[HP:0000822, HP:0000822, HP:0000822, HP:0000822, HP:0032263]","[hypertension, high blood pressure, arterial hypertension, systemic hypertension, increased blood pressure]","[0.9999998211860657, 0.8678926825523376, 0.8463543057441711, 0.8321573734283447, 0.8045231103897095]"
4,655,665,bradycardia,HP,0.9965,HP:0001662,bradycardia,1.0,"[SNOMEDCT_US:48867003||UMLS:C0428977, Fyler:7013||SNOMEDCT_US:49710005||UMLS:C0085610, SNOMEDCT_US:48867003||UMLS:C0428977, SNOMEDCT_US:3424008||SNOMEDCT_US:86651002||UMLS:C0039231||UMLS:C4020868, SNOMEDCT_US:80313002||UMLS:C0030252]","[HP:0001662, HP:0001688, HP:0001662, HP:0001649, HP:0001962]","[bradycardia, sinus bradycardia, brachycardia, tachycardia, heart palpitations]","[1.000000238418579, 0.9310865998268127, 0.7624821066856384, 0.7563266754150391, 0.7407756447792053]"


### JSON Lines

In [10]:
import json

def create_jsonl(records):

    if isinstance(records, str):
        records = [records]

    json_records = []

    for text in records:
        record = {
            "text": text
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

In [11]:
input_jsonl_data = create_jsonl(sample_text)
data = process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": [{"begin": 67, "end": 89, "ner_chunk": "tricuspid regurgitation", "ner_label": "HP", "ner_confidence": "0.9826", "concept_code": "HP:0005180", "resolution": "tricuspid regurgitation", "score": 1.0000003576278687, "aux": ["Fyler:1161||SNOMEDCT_US:111287006||UMLS:C0040961", "Fyler:1161||SNOMEDCT_US:111287006||UMLS:C0040961", "Fyler:1161||SNOMEDCT_US:111287006||UMLS:C0040961", "EPCC:06.01.92||ICD-10:Q22.4||SNOMEDCT_US:49915006||UMLS:C0040963", "EPCC:06.01.00||UMLS:C4025753"], "all_codes": ["HP:0005180", "HP:0005180", "HP:0005180", "HP:0010446", "HP:0001702"], "all_resolutions": ["tricuspid regurgitation", "tricuspid valve regurgitation", "tricuspid insufficiency", "tricuspid stenosis", "abnormality of the tricuspid valve"], "all_score": [1.0000003576278687, 0.9726435542106628, 0.8386023640632629, 0.8336495161056519, 0.8167619705200195]}, {"begin": 197, "end": 211, "ner_chunk": "aortic stenosis", "ner_label": "HP", "ner_confidence": "0.8053", "concept_code": "HP:0001650", "

### B. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [13]:
import json
import os

input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)

validation_json_file_name = "input.json"

validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [14]:
input_jsonl_data = create_jsonl(docs)
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}")

write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}")

### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [16]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):

    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [17]:
process_s3_json_output_and_save(validation_json_file_name)

{'predictions': [[{'begin': 126,
    'end': 134,
    'ner_chunk': 'shortness',
    'ner_label': 'HP',
    'ner_confidence': '0.8931',
    'concept_code': 'HP:0004322',
    'resolution': 'short stature',
    'score': 0.8393914699554443,
    'aux': ['SNOMEDCT_US:237836003||UMLS:C0349588',
     'SNOMEDCT_US:34911001||UMLS:C4551492',
     'UMLS:C0239399',
     'UMLS:C0239399',
     'UMLS:C3150077||UMLS:C4020838'],
    'all_codes': ['HP:0004322',
     'HP:0000054',
     'HP:0009826',
     'HP:0009826',
     'HP:0003502'],
    'all_resolutions': ['short stature',
     'short penis',
     'short limb',
     'short limbs',
     'short stature, mild'],
    'all_score': [0.8393914699554443,
     0.8264893889427185,
     0.8134717345237732,
     0.812436580657959,
     0.8091142177581787]},
   {'begin': 457,
    'end': 468,
    'ner_chunk': 'hypertension',
    'ner_label': 'HP',
    'ner_confidence': '0.995',
    'concept_code': 'HP:0000822',
    'resolution': 'hypertension',
    'score': 0.99999

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [19]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [20]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"predictions": [{"begin": 126, "end": 134, "ner_chunk": "shortness", "ner_label": "HP", "ner_confidence": "0.8931", "concept_code": "HP:0004322", "resolution": "short stature", "score": 0.8393914699554443, "aux": ["SNOMEDCT_US:237836003||UMLS:C0349588", "SNOMEDCT_US:34911001||UMLS:C4551492", "UMLS:C0239399", "UMLS:C0239399", "UMLS:C3150077||UMLS:C4020838"], "all_codes": ["HP:0004322", "HP:0000054", "HP:0009826", "HP:0009826", "HP:0003502"], "all_resolutions": ["short stature", "short penis", "short limb", "short limbs", "short stature, mild"], "all_score": [0.8393914699554443, 0.8264893889427185, 0.8134717345237732, 0.812436580657959, 0.8091142177581787]}, {"begin": 457, "end": 468, "ner_chunk": "hypertension", "ner_label": "HP", "ner_confidence": "0.995", "concept_code": "HP:0000822", "resolution": "hypertension", "score": 0.9999997615814209, "aux": ["SNOMEDCT_US:24184005||SNOMEDCT_US:38341003||UMLS:C0020538||UMLS:C0497247", "SNOMEDCT_US:24184005||SNOMEDCT_US:38341003||UMLS:C0020538|

In [ ]:
model.delete_model()

### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

